How to clean the data

In [ ]:
import pandas as pd

## What the info() method can tell you

In [ ]:
polls = pd.read_csv('https://github.com/fenago/dw/raw/main/data/president_polls_2016.csv')

In [ ]:
polls.info(verbose=True, memory_usage='deep', show_counts=True)

In [ ]:
# NOTE: This may take a long time to run
jobs = pd.read_excel('../../data/oesm18all/all_data_M_2018.xlsx')

In [ ]:
jobs.info(verbose=True, memory_usage='deep', show_counts=True)

## What the unique values can tell you

In [ ]:
polls.nunique()

In [ ]:
polls.apply(pd.unique)

## What the value counts can tell you

In [ ]:
polls.state.value_counts().head(4)

In [ ]:
polls.state.value_counts(normalize=True).head(4)

In [ ]:
polls.multiversions.value_counts(dropna=False)

In [ ]:
polls[['state','grade']].value_counts()

## How to drop rows

In [ ]:
#polls = polls[polls.type == 'now-cast']

In [ ]:
polls = polls.query('type == "now-cast"')

In [ ]:
import sqlite3
fires_con = sqlite3.connect('../../data/Data/FPA_FOD_20170508.sqlite')
fires = pd.read_sql_query(
    '''SELECT STATE, FIRE_YEAR, DATETIME(DISCOVERY_DATE) AS DISCOVERY_DATE,
              FIRE_NAME, FIRE_SIZE, LATITUDE, LONGITUDE
       FROM Fires''', fires_con)
fires.columns = fires.columns.str.lower()

In [ ]:
fires[fires.duplicated(keep=False)]

In [ ]:
fires.drop_duplicates(keep='first', inplace=True)

## How to drop columns

In [ ]:
polls.nunique()

In [ ]:
#polls = polls.drop(columns=['cycle','forecastdate'])

In [ ]:
polls.drop(columns=['cycle','forecastdate'], errors='raise', inplace=True)

## How to rename columns

In [ ]:
polls = polls.rename(columns={
    'rawpoll_clinton':'clinton_pct',
    'rawpoll_trump':'trump_pct'})

In [ ]:
polls_names_dict = {'rawpoll_clinton':'clinton_pct',
                    'rawpoll_trump':'trump_pct'}
polls.rename(columns = polls_names_dict, inplace = True)

In [ ]:
polls.columns = polls.columns.str.replace('_pct','')

In [ ]:
jobs.columns = jobs.columns \
    .str.replace('^a_','annual_') \
    .str.replace('^h_','hourly_') \
    .str.replace('_pct','_percent')

## How to find missing values

In [ ]:
mortality_data = pd.read_csv('https://github.com/fenago/dw/raw/main/mortality_missing_values.csv')

In [ ]:
mortality_data.head(5)

In [ ]:
mortality_data.loc[5:9]

In [ ]:
missing_count = mortality_data.shape[0] - mortality_data.count()
print(missing_count)

In [ ]:
mortality_data[mortality_data.isnull().any(axis=1)]

In [ ]:
mortality_data[mortality_data.DeathRate.isnull()]

In [ ]:
mortality_data[mortality_data.DeathRate.notnull()]

## How to drop rows with missing values

In [ ]:
mortality_data = mortality_data.dropna()
mortality_data.head()

In [ ]:
mortality_data.dropna(thresh=2, inplace=True)

In [ ]:
mortality_data.dropna(subset=['DeathRate'], inplace=True)

## How to fill missing values

In [ ]:
mortality_data = pd.read_csv('mortality_missing_values.csv')
mortality_data.head()

In [ ]:
mortality_data.DeathRate.fillna(value=mortality_data.DeathRate.mean(), inplace=True)
mortality_data.head(4)

In [ ]:
mortality_data = pd.read_csv('mortality_missing_values.csv')
mortality_data.fillna(method = 'ffill', limit=2, inplace=True)
mortality_data.head(4)

In [ ]:
mortality_data = pd.read_csv('mortality_missing_values.csv')
mortality_data = mortality_data.interpolate()
mortality_data.head(4)

## How to find date and number columns that are imported as objects

In [ ]:
polls.select_dtypes('object').head(2)

In [ ]:
jobs.select_dtypes('object').head(2)

## How to convert date and time strings to the datetime data type

In [ ]:
date_cols = ['startdate','enddate','createddate','timestamp']
polls[date_cols].head(2)

In [ ]:
polls[date_cols] = polls[date_cols].apply(pd.to_datetime)
polls[date_cols].head(2)

In [ ]:
polls['startdate'] = polls.startdate.dt.strftime("%m/%d/%Y")
polls['enddate'] = polls.enddate.dt.strftime("%m-%d-%y")
polls[date_cols].head(2)

## How to convert object columns to numeric data types

In [ ]:
# pd.to_numeric(jobs.tot_emp)

In [ ]:
jobs.tot_emp.tail(3)

In [ ]:
pd.to_numeric(jobs.tot_emp, errors='coerce').tail(3)

## How to work with the category data type

In [ ]:
fires.state = fires.state.astype('category')

In [ ]:
statCounts = pd.read_stata('../../data/GSS7218_R3.DTA', columns=['year','wrkstat'])

In [ ]:
filteredStatCounts = statCounts.query(
    'wrkstat in ["working fulltime","working parttime","retired"]')
filteredStatCounts.wrkstat.value_counts()

In [ ]:
# produces a warning that you'll learn about in chapter 7
filteredStatCounts['wrkstat'] = \
    filteredStatCounts.wrkstat.cat.remove_unused_categories()
filteredStatCounts.wrkstat.value_counts()

## How to replace invalid values and convert a column’s data type

In [ ]:
import numpy as np
jobs.tot_emp.replace(to_replace=['*','**'], value=[np.nan,np.nan]) # ,inplace=True)

In [ ]:
jobs.tot_emp.replace({'*':np.nan, '**':np.nan}, inplace=True)

In [ ]:
jobs.hourly_median = jobs.hourly_median.replace(to_replace='#', value = 100)

In [ ]:
jobs.hourly.replace(to_replace=np.nan, value=False, inplace=True)

## How to fix data type problems when you import the data

In [ ]:
date_cols = ['forecastdate','startdate','enddate','createddate','timestamp']

In [ ]:
polls = pd.read_csv('../../data/president_polls_2016.csv')
polls[date_cols].head(3)

In [ ]:
polls_new = pd.read_csv('../../data/president_polls_2016.csv', parse_dates=date_cols)
polls_new[date_cols].head(3)

## How to find outliers

In [ ]:
mortality_data = pd.read_pickle('mortality_cleaned.pkl')

In [ ]:
mortality_group = mortality_data.query('AgeGroup == "15-19 Years"')
mortality_group.head(3)

In [ ]:
# save DataFrame for later use
mortality_group.to_pickle('mortality_group.pkl')

In [ ]:
mortality_group.describe().T

In [ ]:
mortality_group.plot(x='Year', y='DeathRate')

In [ ]:
mortality_group.query('DeathRate > 500')

## How to fix outliers

In [ ]:
mortality_group = pd.read_pickle('mortality_group.pkl')

mortality_group = mortality_group.query('DeathRate <= 500')

In [ ]:
mortality_group = pd.read_pickle('mortality_group.pkl')

mortality_group.loc[mortality_group.DeathRate > 500, 'DeathRate'] = 450

In [ ]:
mortality_group = pd.read_pickle('mortality_group.pkl')

mortality_group.loc[mortality_group.DeathRate > 500, 'DeathRate'] \
    = mortality_group.DeathRate.mean()
mortality_group.plot(x='Year', y='DeathRate')

In [ ]:
mortality_group = pd.read_pickle('mortality_group.pkl')

mortality_group.loc[mortality_group.Year.isin([1917,1918,1919,1920]),
                    'DeathRate'] = None
mortality_group = mortality_group.interpolate()
mortality_group.plot(x='Year', y='DeathRate')